# 7.2　《分析》データを利用してクレジットカードの顧客を分類し、特徴を把握しよう
## 7.2.2　データの理解

#### コード7.1


In [ ]:
! pip install japanize-matplotlib

#### コード7.2


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

#### コード7.3


In [ ]:
cc_cust = pd.read_csv('CC_GENERAL.csv')

#### コード7.4


In [ ]:
cc_cust.head()

#### コード7.5


In [ ]:
cc_cust.shape

#### コード7.6


In [ ]:
cc_cust.isnull().sum()

#### コード7.7


In [ ]:
cc_cust.info()

#### コード7.8


In [ ]:
cc_cust.describe()

#### コード7.9


In [ ]:
# sweetvizが動作しない場合は、以下の行の「#」を削除して、再実行してください
#! pip install numpy==1.26.4
! pip install sweetviz

#### コード7.10


In [ ]:
import sweetviz as sv
sv.config_parser.read_string("[General]\nuse_cjk_font=1")

report = sv.analyze(cc_cust)
report.show_notebook()

## 7.2.3　データの前処理

#### コード7.11


In [ ]:
cc_cust[cc_cust['クレジット限度額'].isnull()]

#### コード7.12


In [ ]:
sns.displot(data=cc_cust, x='クレジット限度額', kde=True, bins=50)
plt.xlabel('クレジット限度額')
plt.ylabel('顧客数')
plt.show()

#### コード7.13


In [ ]:
cc_cust['クレジット限度額'] = cc_cust['クレジット限度額'].fillna(cc_cust['クレジット限度額'].median())

#### コード7.14


In [ ]:
cc_cust[cc_cust['最低支払額'].isnull()].describe()

#### コード7.15


In [ ]:
cc_cust['最低支払額'] = cc_cust['最低支払額'].fillna(0)

#### コード7.16


In [ ]:
cc_cust.isnull().sum()

#### コード7.17（標準化）

In [ ]:
clustering_cols = [col for col in cc_cust.columns if col != '顧客ID']
clustering_cols

#### コード7.18


In [ ]:
X = cc_cust[clustering_cols]
X.head()

#### コード7.19


In [ ]:
sc = StandardScaler()
sc.fit(X)
X_sc = sc.transform(X)
X_sc = pd.DataFrame(X_sc, columns=X.columns)
X_sc.head()

#### コード7.20


In [ ]:
X_sc.describe()

## 7.2.4　モデリング 〜*K*-meansによるクラスタリング〜

#### コード7.21


In [ ]:
n_clusters=15
inertia=[]
for i in range(1,n_clusters):
    kmean= KMeans(n_clusters=i,random_state=123, init='k-means++')
    kmean.fit(X_sc)
    inertia.append(kmean.inertia_)

#### コード7.22


In [ ]:
#import numpy as np
plt.figure(1 , figsize = (8 ,6))
plt.plot(np.arange(1 , n_clusters) , inertia , 'o')
plt.plot(np.arange(1 , n_clusters) , inertia , '-' , alpha = 0.5)
plt.xlabel('Number of Clusters') , plt.ylabel('Inertia')
plt.show()

#### コード7.23


In [ ]:
kmean = KMeans(n_clusters=5)
kmean.fit(X_sc)
labels = kmean.labels_
clusters=pd.concat([cc_cust, pd.DataFrame({'クラスターラベル':labels})], axis=1)
clusters.head()

## 7.2.5　クラスタリングのアウトプットを解釈・評価しよう

#### コード7.24


In [ ]:
cluster_sizes = clusters.groupby('クラスターラベル')['顧客ID'].count().reset_index().rename(columns={'顧客ID': '顧客数'})
clusters_excluded_id = clusters.drop(columns=['顧客ID'])
cluster_means = clusters_excluded_id.groupby('クラスターラベル').mean().reset_index()
cluster_summary = pd.merge(cluster_sizes, cluster_means, on='クラスターラベル', how='inner')

cluster_summary.head()

#### コード7.25


In [ ]:
styled_df = cluster_summary.style.background_gradient(cmap='Blues', axis=0)
styled_df